__Content Based Recommendation System :__ Promote movies based on the content of the movies the user has watched before

__Popularity Based Recommendation System:__ recommend movies based on popularity of movies

__Collaborative Recommendation System:__ recommend movies based on watching pattern of various users

In [1]:
# Steps
# import data
# Data pre-procssing
# Feature extraction -> User input -> Cosine Similarities(Used to find the similarities between vectors)

# List of movies

In [4]:
# Import libraries
import numpy as np
import pandas as pd
import difflib # For comparing movies (we will find a close match between the user input and available movies)
from sklearn.feature_extraction.text import TfidfVectorizer # Use to convert textual data into numbers
from sklearn.metrics.pairwise import cosine_similarity # To find the similarity or cosine score between the movies, 
# High similarity scocre means those movies are similar, so we can recommend them.

In [5]:
### Data collection and processing

# loading the data from the csv file to pandas dataframe
movies_data = pd.read_csv("movies.csv")

In [6]:
pd.set_option('display.max_columns',1000)

In [3]:
# printing the first five rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [7]:
movies_data.title[30]

'Spider-Man 2'

In [8]:
movies_data.tagline[0] 

'Enter the World of Pandora.'

In [9]:
# number of rows and columns in the data frame
movies_data.shape

(4803, 24)

In [12]:
# selecting the relevant feature for recommendation
selected_features = ["genres", "keywords", "tagline", "cast", "director"] 
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [13]:
movies_data[selected_features]

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,"He didn't come looking for trouble, but troubl...",Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,NaN,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,NaN,NaN,A New Yorker in Shanghai,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [16]:
movies_data[selected_features]

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,"He didn't come looking for trouble, but troubl...",Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,,A newlywed couple's honeymoon is upended by th...,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,,,A New Yorker in Shanghai,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [15]:
# replacing the null values with null string
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna("")   

In [10]:
# combining all the 5 selected features
combined_features = movies_data["genres"] + " "+ movies_data["keywords"]+ " "+ movies_data["tagline"]+ " "+movies_data["cast"]+ " "+movies_data["director"]

In [11]:
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [12]:
# converting the text data to feature vectors (The cosine similarity works best with numeric data)
vectorizer = TfidfVectorizer()

In [13]:
features_vectors = vectorizer.fit_transform(combined_features)

In [14]:
print(features_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

### Cosine Similarity
Finding the similaririty score(confidence value), numerical score value for all the different movies

In [16]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(features_vectors)

In [17]:
similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [18]:
print(similarity.shape)

(4803, 4803)


In [21]:
# Getting the movie name from the user
movie_name = input("Enter your favorite movie name : ")

Enter your favorite movie name : spectre


In [20]:
# creating a list with all the movie names given in the dataset
# This will contain all the movies name in the dataset for comparism with the input given
list_of_all_titles = movies_data["title"].tolist()
list_of_all_titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [22]:
# finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
find_close_match

['Spectre', 'Suspect Zero', 'Sphere']

In [23]:
# Picking the first value in the list as my close match
close_match = find_close_match[0]
print(close_match)

Spectre


In [109]:
# Break downn of finding the index of the movie with the close match title
#movies_data[movies_data.title == close_match]["index"].values[0]

In [25]:
# Finding the index of the movie with title
# Checking which particular row the close match(iron man), for its similarities
index_of_the_movie = movies_data[movies_data.title == close_match]["index"].values[0]

In [26]:
print(index_of_the_movie)

2


In [30]:
list_1 = ["Victor", 2, 3, 4, 5, 6, 7, 8, 9]
for i in enumerate(list_1):
    print(i)

(0, 'Victor')
(1, 2)
(2, 3)
(3, 4)
(4, 5)
(5, 6)
(6, 7)
(7, 8)
(8, 9)


In [27]:
print(similarity[index_of_the_movie])

[0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]


In [31]:
# Getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [32]:
# The first number is the index of the movies and the second number is the similarity score between the input movie and other movies
print(similarity_score)

[(0, 0.03773299957717928), (1, 0.032814990806793745), (2, 1.0000000000000002), (3, 0.05179498007149405), (4, 0.05772600875531061), (5, 0.05423729174306449), (6, 0.008427651382813465), (7, 0.07258270361531877), (8, 0.03484297090109418), (9, 0.04026640889893438), (10, 0.05142951922781709), (11, 0.13968966431641786), (12, 0.014955647747981473), (13, 0.013502872285510087), (14, 0.04056565208446879), (15, 0.050564960649271434), (16, 0.03907993864609115), (17, 0.014306813289387526), (18, 0.006039100988903486), (19, 0.012702128925458677), (20, 0.014586513752027637), (21, 0.012292681787809338), (22, 0.007520120280294526), (23, 0.07651285556154855), (24, 0.013818330026549261), (25, 0.011467364105495693), (26, 0.04763384483049542), (27, 0.01334296746711279), (28, 0.014649003679301262), (29, 0.3236516681724619), (30, 0.08408864917048521), (31, 0.025587599816574425), (32, 0.04899880866732916), (33, 0.04095824554793068), (34, 0.0), (35, 0.015423653867325422), (36, 0.043385197976649165), (37, 0.0330

In [33]:
# len of the list
len(similarity_score)

4803

In [34]:
# We want only the movies with highest similarity values
# We need to sort the list from the highest to the lowest similarity score

sorted_similar_movies = sorted(similarity_score, key= lambda x:x[1], reverse = True)

In [38]:
movies_data.title[153]

'Mission: Impossible - Ghost Protocol'

In [36]:
sorted_similar_movies

[(2, 1.0000000000000002),
 (29, 0.3236516681724619),
 (153, 0.155667043773143),
 (1100, 0.1416059256064831),
 (11, 0.13968966431641786),
 (3541, 0.13890728056362578),
 (339, 0.13873738560364413),
 (252, 0.1299391327135077),
 (497, 0.12937925196871133),
 (103, 0.12689946413168623),
 (62, 0.12613248158601614),
 (3104, 0.12403749160808117),
 (1940, 0.12151641974871197),
 (1305, 0.12042968521137337),
 (3336, 0.12039444578653513),
 (287, 0.11212756532905882),
 (3631, 0.111149619452692),
 (1769, 0.11052822183365212),
 (571, 0.10909429158114965),
 (354, 0.10738129191665335),
 (113, 0.10716172298060986),
 (3047, 0.10664858758680666),
 (444, 0.10559169021859009),
 (1993, 0.10480264416307354),
 (3242, 0.10399391010863969),
 (782, 0.10255610312814128),
 (529, 0.10206873935621588),
 (695, 0.10192331974758115),
 (219, 0.10084686928392199),
 (206, 0.10076137144012465),
 (3415, 0.09991790824937988),
 (433, 0.09962983420479513),
 (249, 0.09879054608585564),
 (3352, 0.0982665590305202),
 (1999, 0.09738

In [39]:
# print the names of similar movies based on the index

print("Movies suggested for you : \n")

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]["title"].values[0]
    if (i < 5):
        print(i, ".", title_from_index)
        i += 1

Movies suggested for you : 

1 . Spectre
2 . Skyfall
3 . Mission: Impossible - Ghost Protocol
4 . Johnny English Reborn


### Movie Recommandation System

In [40]:
movie_name = input("Enter your favorite movie name : ")

list_of_all_titles = movies_data["title"].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]["index"].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key= lambda x:x[1], reverse = True)

print("Movies suggested for you : \n")

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]["title"].values[0]
    if (i < 30):
        print(i, ".", title_from_index)
        i += 1

Enter your favorite movie name : spectre
Movies suggested for you : 

1 . Spectre
2 . Skyfall
3 . Mission: Impossible - Ghost Protocol
4 . Johnny English Reborn
5 . Quantum of Solace
6 . Irreversible
7 . The Incredibles
8 . The Green Hornet
9 . Red Dragon
10 . The Sorcerer's Apprentice
11 . The Legend of Tarzan
12 . Coriolanus
13 . Carnage
14 . Water for Elephants
15 . Diamonds Are Forever
16 . Django Unchained
17 . Heavenly Creatures
18 . Spy Kids: All the Time in the World
19 . Inglourious Basterds
20 . The Girl with the Dragon Tattoo
21 . Harry Potter and the Order of the Phoenix
22 . Big Eyes
23 . Road to Perdition
24 . Final Destination
25 . Escape from Alcatraz
26 . The Spirit
27 . Tears of the Sun
28 . GoldenEye
29 . Charlie's Angels: Full Throttle
